In [2]:
import geopandas as gpd
import pandas as pd
import os

In [3]:
os.chdir('/project/cper_neon_aop/cper_pdog_uas')

In [4]:
gdf_ground_points_all = gpd.read_file('ground_data/cper_pdog_points_2021Sept.shp')

for idx, pasture in enumerate(['5W', '22W', '22E', '29-30', 'CN']):
    if idx == 0:
        df_bboxes = pd.read_csv('train_tiles/train_bboxes_' + pasture + '.csv')
    else:
        df_bboxes = pd.concat([df_bboxes, pd.read_csv('train_tiles/train_bboxes_' + pasture + '.csv')])

df_bboxes = df_bboxes[df_bboxes['Digitize'] == 1].reset_index(drop=True)
df_bboxes['path_pre'] = df_bboxes.apply(lambda x: os.path.join('train_tiles',
                                                           x['Pasture'] + '_' + x['Type'],
                                                           x['Pasture'] + '_' + x['Tile'] + '_'), axis=1)
df_bboxes.loc[df_bboxes[df_bboxes['Type'] == 'random'].groupby(
    'Pasture').sample(n=2, random_state=123).index, 'trainer'] = 'All'
df_bboxes.loc[df_bboxes[df_bboxes['Type'] == 'burrows_active'].groupby(
    'Pasture').sample(n=1, random_state=223).index, 'trainer'] = 'All'

df_bboxes_sub = df_bboxes.loc[df_bboxes['trainer'] != 'All'].sample(frac=1, random_state=323)

df_bboxes_sub.iloc[:30, df_bboxes_sub.columns.get_loc('trainer')] = 'Lauren'
df_bboxes_sub.iloc[30:60, df_bboxes_sub.columns.get_loc('trainer')] = 'David'
df_bboxes_sub.iloc[60:90, df_bboxes_sub.columns.get_loc('trainer')] = 'Sean'
df_bboxes_sub.iloc[90:, df_bboxes_sub.columns.get_loc('trainer')] = 'Nick'

df_bboxes_fnl = pd.concat([df_bboxes[df_bboxes['trainer'] == 'All'], df_bboxes_sub])
df_bboxes_fnl.loc[df_bboxes_fnl['Poly_ID'].isnull(), 'Poly_ID'] = 'None'
type_dict = {'burrows_active': 0, 'random': 1}

In [6]:
df_bboxes_fnl.to_csv('train_tiles/train_bboxes_all_assigned.csv')